<a href="https://colab.research.google.com/github/SVis760/small-projects-at-random/blob/main/SPARQL_query_API_to_LeafLet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **POC gebruik SPARQL API in LeafLet map**

Script gebouwd samen met mijn buddy chatGTP.
API van de query: [Aardbevingen in Groningen](https://linkeddata.cultureelerfgoed.nl/rce/-/queries/Query-17/4) gebruikt.

Download de html output als je het runt, en dubbelklik. Maar ik zal opsturen.

Idee was om te kijken of zou lukken om API direct te gebruiken en wat er allemaal bij komt kijken en of LeafLet een goede oplossing is voor LOD.

Zoals je ziet moeten de geo WKT omgezet worden naar format dat LeafLet begrijpt.
De Rietveld data map had ik moeite mee met laden, ik ga kijken of ik daar nog iets aan kan doen. Anders hoor ik graag ideeen!

In [ ]:
from shapely import wkt
from shapely.geometry import Polygon, MultiPolygon, Point

In [ ]:
import requests, json
url = "https://api.linkeddata.cultureelerfgoed.nl/queries/rce/Query-17/4/run"
response = requests.get(url)
data = response.json()
#data

In [ ]:

def modify_list_with_wkt(data):
    # Define a function to convert WKT to lat/lng format
    def wkt_to_lat_lng(wkt_string):
        # Convert WKT string to a Shapely geometry object
        geometry = wkt.loads(wkt_string)

        # If it's a MultiPolygon, convert the first polygon to a point
        if isinstance(geometry, MultiPolygon):
            first_polygon = geometry.geoms[0]
            point = first_polygon.representative_point()
        # If it's a Polygon, convert it to a point
        elif isinstance(geometry, Polygon):
            point = geometry.representative_point()
            # If it's a Point, extract latitude and longitude directly
        elif geometry.geom_type == 'Point':
            lat = geometry.y
            lng = geometry.x
            return {"lat": lat, "lng": lng}
        else:
            return None  # Unsupported geometry type

        # Extract latitude and longitude from the point
        lat = point.y
        lng = point.x

        return {"lat": lat, "lng": lng}

    # Modify the list data to include lat/lng format
    modified_data = []
    for item in data:
        if "shape_rm" in item:
            wkt_point = item["shape_rm"]
            lat_lng = wkt_to_lat_lng(wkt_point)
            if lat_lng:
                item["lat"] = lat_lng["lat"]
                item["lng"] = lat_lng["lng"]
                del item["shape_rm"]  # Remove the original WKT point
                modified_data.append(item)

    return modified_data


result = modify_list_with_wkt(data)
print(result)


[{'rm': 'https://linkeddata.cultureelerfgoed.nl/cho-kennis/id/rijksmonument/11748', 'rm_n': '12323', 'aard': 'https://data.cultureelerfgoed.nl/term/id/rn/fc966a68-8863-4970-a83e-110f96006c21', 'locatieNaam_ab': 'Pijpplein to 6, Farmsum', 'shape_rmColor': 'purple', 'mapEndpoint': 'https://haleconnect.com/ows/services/org.874.7e01e60c-8887-425c-af9b-e2cf6af9181b_wms?SERVICE=WMS&Request=GetCapabilities', 'lat': 53.3226922859735, 'lng': 6.928261587174925}, {'rm': 'https://linkeddata.cultureelerfgoed.nl/cho-kennis/id/rijksmonument/11748', 'rm_n': '12323', 'aard': 'https://data.cultureelerfgoed.nl/term/id/rn/fc966a68-8863-4970-a83e-110f96006c21', 'locatieNaam_ab': 'Pijpplein to 6, Farmsum', 'shape_rmColor': 'purple', 'mapEndpoint': 'https://haleconnect.com/ows/services/org.874.7e01e60c-8887-425c-af9b-e2cf6af9181b_wms?SERVICE=WMS&Request=GetCapabilities', 'lat': 53.3226922859735, 'lng': 6.928261587174925}, {'rm': 'https://linkeddata.cultureelerfgoed.nl/cho-kennis/id/rijksmonument/53093', 'rm_

In [ ]:
# Convert result to JSON string
result_json = json.dumps(result)

# HTML content with result injected into JavaScript
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Leaflet Map with JSON Data</title>
    <link rel="stylesheet" href="https://unpkg.com/leaflet/dist/leaflet.css"/>
    <style>
        #map {
            height: 1000px;
        }
    </style>
</head>
<body>

<div id="map"></div>

<script src="https://unpkg.com/leaflet/dist/leaflet.js"></script>
<script>
    var map = L.map('map').setView([53.2191, 6.56667], 10);

    L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
        maxZoom: 19,
    }).addTo(map);

    // Sample JSON data with coordinates
    var jsonData = %s;

    // Add markers from JSON data
    jsonData.forEach(function (marker) {
        L.marker([marker.lat, marker.lng]).addTo(map)
            .bindPopup(marker.rm_n);
    });
</script>

</body>
</html>
""" % result_json

# Write HTML content to a file
with open("leaflet_map.html", "w") as file:
    file.write(html_content)
